# Recommendation: Scoprire Propensioni d'Acquisto di Utenti con l'Algebra Lineare

**Programmazione di Applicazioni Data Intensive**  
Laurea in Ingegneria e Scienze Informatiche  
DISI - Università di Bologna, Cesena

Proff. Gianluca Moro, Roberto Pasolini  
`nome.cognome@unibo.it`

## Recommendation: Prevedere le Propensioni di Acquisto <br>e Molto Altro ... 

- Ogni azienda ha i dati storici di acquisto di ciascun cliente/utente
 - usiamo un sottoinsieme di dati reali presi da AMAZON
- Vogliamo **raccomandare/suggerire ai singoli utenti quali prodotti acquistare** 
 - idea: utenti con storie di acquisti simili, è probabile che faranno acquisti simili anche in futuro 
  - metodo: proponiamo ad ogni utente gli acquisti fatti da altri utenti con storico più simile al proprio
- con semplici **operazioni tra matrici** otteniamo una soluzione efficace in una manciata di righe di codice

- Per le operazioni ci avvaliamo di NumPy, che importiamo con l'alias convenzionale `np`

In [1]:
import numpy as np

- Metodi più sofisticati di *recommendation* saranno oggetto di una lezione successiva
- Questi sono **metodi generali e applicabili a numerosi problemi simili**
 - prevedere per ogni utente le notizie di interesse, l'orientamento politico, il prossimo lavoro, le prossime vacanze, l'opinione su temi sociali etc.  

## Matrice degli Acquisti

- Nel file `estore-purchases-2000.npy.gz` è contenuta una **"matrice degli acquisti" binaria**, basata su dati fino alla fine del 2000
  - ogni **riga** corrisponde ad un **utente** con almeno 30 acquisti
  - ogni **colonna** corrisponde ad un **prodotto** distinto nel database, acquistato da almeno uno degli utenti
  - il valore della cella _i,j_ è **1** se l'utente _i_ ha acquistato il prodotto _j_ prima della data considerata, **0** altrimenti

## Funzione per il Caricamento delle Matrici

- I file contenenti le matrici usate per l'esercitazione sono disponibili online
  - Tramite il modulo `urrlib` della libreria standard di Python è possibile, dato l'URL del file, caricarlo dall'interprete senza scaricare manualmente il file
- Le matrici sono contenute in file `.npy`, un formato specifico di NumPy: usiamo la funzione `load` per caricarle
  - per salvare un array in questo formato si usa `np.save(file, array)`
  - i file sono compressi in GZIP, usiamo il modulo `gzip` della libreria standard per leggerli
- Definiamo una funzione per eseguire tutte queste operazioni in automatico per ciascuna matrice da caricare

In [2]:
import gzip
def load_matrix(zipped_file):
    with gzip.open(zipped_file, "rb") as unzipped_file:
        return np.load(unzipped_file)

## Caricamento della Matrice

- Usiamo la funzione appena definita per caricare la matrice degli acquisti

In [3]:
purchases = load_matrix("estore-purchases-2000.npy.gz")

- Visualizziamo una porzione della matrice caricata...

In [4]:
purchases[-5:, :20]   # ultime 5 righe, prime 20 colonne

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int16)

- Ad es. il penultimo cliente (4a riga) ha acquistato tra gli altri il 12° e 13° prodotto

## Dimensioni della Matrice

- La _forma_ della matrice indica il suo numero di righe (utenti) e di colonne (prodotti)
- Tramite l'_unpacking_, assegnamo i due numeri a due variabili distinte

In [5]:
n_users, n_items = purchases.shape

- Quanti sono gli utenti e i prodotti distinti ?

In [6]:
print("{} utenti, {} prodotti".format(n_users, n_items))

178 utenti, 3384 prodotti


## Totali Acquisti

- Col metodo `sum` possiamo effettuare la somma per righe e per colonne della matrice per individuare
  - il numero totale di prodotti acquistati da ciascun utente
  - il numero totale di utenti che hanno acquistato ciascun prodotto

In [7]:
users_purchases = purchases.sum(1)   # somma tutte le colonne
items_purchases = purchases.sum(0)   # somma tutte le righe

- Possiamo verificare che ogni utente considerato abbia almeno 30 acquisti

In [8]:
users_purchases.min()

30

- Qual è il numero medio di prodotti acquistati per utente ?

In [9]:
mean_purchases = users_purchases.sum() / n_users
mean_purchases

54.39887640449438

- Qual è il numero max di prodotti acquistati da un qualche utente ?

In [10]:
users_purchases.max()

242

_**Quesito:** quanti utenti hanno acquistato il prodotto più venduto?_

## Similarità tra Utenti

- Vogliamo suggerire prodotti agli utenti in base a **cos'hanno acquistato utenti simili**
- Come determinare quanto due utenti siano "simili"?
- Possiamo contare **quanti sono i prodotti che entrambi hanno acquistato**
- Questo numero è dato dal **prodotto scalare delle righe** corrispondenti nella matrice degli acquisti
  - questo è infatti la somma di ogni prodotto di valori corrispondenti, che è 1 dove entrambe le righe hanno 1 e 0 altrimenti
- Ad es., il numero di acquisti in comune tra il primo e il secondo utente è ...

In [11]:
purchases[0].dot(purchases[1])

4

### Matrice di Similarità

- Per ottenere i prodotti scalari tra tutte le righe, possiamo **moltiplicare la matrice per la sua trasposta**
  - si moltiplicano le righe della matrice originale per le colonne della trasposta, che corrispondono sempre alle righe dell'originale

In [12]:
common_purchases = purchases.dot(purchases.T)

- Otteniamo una matrice quadrata simmetrica, di ordine pari al numero di utenti

In [13]:
# la matrice è uguale alla sua trasposta ?
np.array_equal(common_purchases, common_purchases.T)

True

In [14]:
common_purchases.shape

(178, 178)

- Visualizziamo un estratto della matrice...

In [15]:
common_purchases[:10, :10]

array([[38,  4,  1,  0,  1,  2,  1,  1,  0,  2],
       [ 4, 32,  1,  0,  0,  2,  0,  1,  2,  4],
       [ 1,  1, 50,  1,  4,  2,  1,  2,  1,  7],
       [ 0,  0,  1, 41,  1,  1,  3,  1,  1,  2],
       [ 1,  0,  4,  1, 34,  1,  0,  2,  1,  2],
       [ 2,  2,  2,  1,  1, 47,  1,  0,  2,  1],
       [ 1,  0,  1,  3,  0,  1, 72,  1,  1,  2],
       [ 1,  1,  2,  1,  2,  0,  1, 40,  1,  6],
       [ 0,  2,  1,  1,  1,  2,  1,  1, 33,  3],
       [ 2,  4,  7,  2,  2,  1,  2,  6,  3, 51]], dtype=int16)

- ...possiamo ad esempio notare che
  - tra i primi due utenti ci sono 4 oggetti acquistati in comune
  - il primo ed il quarto utente non hanno alcun acquisto in comune

### Diagonale della Matrice

- La diagonale della matrice creata contiene i prodotti scalari di ciascuna riga della matrice originale con se stessa
- Si può verificare che sono uguali alla somma delle colonne calcolata sopra

In [16]:
np.array_equal(common_purchases.diagonal(), users_purchases)

True

- Prima di procedere, **impostiamo a 0 tutti i valori sulla diagonale** per far sì che tra i simili di ciascun utente non sia incluso lui stesso

In [17]:
# fill_diagonal imposta tutti gli elementi della diagonale al valore dato
np.fill_diagonal(common_purchases, 0)
common_purchases[:5, :5]

array([[0, 4, 1, 0, 1],
       [4, 0, 1, 0, 0],
       [1, 1, 0, 1, 4],
       [0, 0, 1, 0, 1],
       [1, 0, 4, 1, 0]], dtype=int16)

_**Quesito:** qual è il massimo numero di prodotti in comune tra due utenti?_

## Stimare il Potenziale Interesse nei Prodotti

- Vogliamo stimare **quanto ciascun utente sia potenzialmente interessato** in ciascun prodotto non ancora acquistato
- Possiamo stimarlo in base a quanto il prodotto sia stato acquistato da utenti simili
- Moltiplichiamo la matrice degli acquisti comuni (che indica la similarità tra utenti) per quella degli acquisti: (utenti x utenti) x (utenti x prodotti) = utenti x prodotti

In [18]:
interest = common_purchases.dot(purchases)

- La forma della matrice è pari a quella della matrice degli acquisti (utenti x prodotti)

In [19]:
interest.shape == purchases.shape

True

- Visualizziamo un estratto della matrice...

In [20]:
interest[0:10, 0:10]

array([[ 0,  5,  0,  4,  4,  3,  1,  0,  4,  3],
       [ 3,  7,  1,  6,  1,  1,  2,  1,  2,  0],
       [ 6,  9,  0, 12,  0,  0,  1,  0,  9,  0],
       [ 1,  2,  2,  8,  0,  0,  0,  2,  5,  2],
       [ 3,  2,  0,  5,  0,  0,  0,  0,  2,  0],
       [ 2,  5,  0, 10,  0,  0,  1,  0,  5,  0],
       [ 0,  5,  5,  5,  1,  2,  1,  5,  2,  3],
       [ 1,  5,  0,  7,  0,  0,  0,  0,  0,  3],
       [ 2,  4,  0,  9,  0,  0,  1,  0,  4,  4],
       [ 6, 17,  0, 25,  0,  0,  0,  0,  9,  2]], dtype=int16)

- Ad esempio si ritiene che il 10° utente sia molto interessato al 4° prodotto

### Scartare i Prodotti già Acquistati

- Nella matrice calcolata `interest`, sono "interessanti" anche i prodotti **già acquistati** da ciascun utente
- Prima di proseguire, **azzeriamo i loro valori** in modo da considerare solo prodotti nuovi
- Convertiamo la matrice degli acquisti iniziale in forma booleana, per ottenere una "maschera"
  - `astype` copia un array convertendone tutti i valori in un altro tipo
  - come in Python standard, gli 0 sono convertiti in `False` e gli altri valori in `True`

In [21]:
purchases_mask = purchases.astype(np.bool)

- Utilizziamo quindi la maschera per **selezionare solo i valori corrispondenti** nella matrice dell'interesse all'acquisto che **impostiamo a 0**

In [22]:
interest[purchases_mask] = 0

## Ottenere _N_ Suggerimenti di Acquisto per ogni Utente

- Da migliaia di prodotti nel catalogo, vogliamo suggerirne **un numero limitato ad ogni utente** massimizzando la probabilità di acquisto
- Fissiamo un numero _N_ di prodotti da suggerire...

In [23]:
N = 20

- ...vogliamo selezionare per ogni utente gli _N_ prodotti con "potenziale interesse" maggiore

### Suggerimenti Basati sulla Previsione degli Interessi 

- Possiamo ottenere suggerimenti migliori sfruttando l'informazione sull'interesse stimato degli utenti verso i prodotti ?
- Per verificarlo, **selezioniamo gli _N_ prodotti di maggiore interesse stimato per ciascun utente**
- Procediamo come segue:
  - assegniamo a ciascun prodotto un "ranking" in base all'interesse per ciascun utente
  - selezioniamo per ogni cliente gli _N_ prodotti col ranking migliore
- Una volta estratti i suggerimenti, ne valuteremo l'accuratezza basandoci sugli acquisti che gli utenti hanno fatto in seguito

### Estrarre l'ordine dei valori: il metodo `argsort`

- Il metodo `argsort` su un vettore restituisce un vettore con i suoi **indici** (da 0 a N-1) **ordinati** secondo i valori

In [24]:
# sia dato il vettore...
x = np.array([32, 8, 2, 4, 16, 64, 1])
# ...il risultato di argsort è...
x.argsort()

array([6, 2, 3, 1, 4, 0, 5])

- Significa che l'elemento minimo è quello di indice 6 (1), seguito da quello di indice 2 (2) e così via fino al massimo di indice 5 (64)
- Nel caso di una matrice, l'operazione è eseguita **lungo una dimensione** a scelta (riga per riga o colonna per colonna)

In [25]:
np.array([[32,  8,  2,  4, 16, 64,  1],
          [ 8, 16,  4, 64, 32,  1,  2]]).argsort(1)   # 1 = riga per riga

array([[6, 2, 3, 1, 4, 0, 5],
       [5, 6, 2, 0, 1, 4, 3]])

### Calcolare il Ranking dei Prodotti

- Applicando l'operazione `argsort` due volte otteniamo il "ranking" dei dati, ovvero l'indice che ogni elemento avrebbe nell'array ordinato
  - Normalmente otteniamo quindi un array che associa 0 all'elemento più basso, 1 al secondo e così via

In [26]:
np.array([32, 8, 2, 4, 16, 64, 1]).argsort().argsort()

array([5, 3, 1, 2, 4, 6, 0])

- D'altra parte, invertendo l'array a cui è applicata l'operazione, otteniamo un array che associa 0 a partire dal valore più alto
- Applichiamo queste operazioni riga per riga all'array `interest`

In [27]:
interest_ranking = (-interest).argsort(1).argsort(1)

### Selezionare i Prodotti da Suggerire

- Abbiamo così ottenuto un array dove per ogni utente (riga) abbiamo i prodotti numerati univocamente da 0 a _N_-1 in ordine di interesse
- Da questo array possiamo quindi selezionare i **valori minori di _N_** per ottenere gli **_N_ prodotti di maggiore interesse** per ciascun utente
  - otteniamo una matrice di valori bool (True/False), che convertiamo in numerica con `astype`

In [28]:
suggestions = (interest_ranking < N).astype(np.int16)

- Questa è la **matrice dei prodotti suggeriti**, che associa ad ogni utente gli _N_ prodotti a cui è potenzialmente più interessato

## Accuratezza dei Suggerimenti di Acquisto

- Come valutare se i suggerimenti ottenuti in questo modo siano azzeccati ?
- Una possibilità consiste nel verificare **se gli oggetti suggeriti siano stati effettivamente acquistati** in un successivo momento
- Nel file `estore-purchases-2014.npy.gz` è fornita una seconda matrice degli acquisti aggiornata
  - righe e colonne sono corrispondenti agli stessi utenti e prodotti della prima matrice
  - i valori sono basati invece su tutti gli acquisti, anche quelli effettuati dopo il 2000
- Possiamo quindi confrontare i prodotti suggeriti con questa nuova matrice

In [29]:
purchases_updated = load_matrix("estore-purchases-2014.npy.gz")

- Verifichiamo che la forma coincida con quella della precedente

In [30]:
purchases_updated.shape == purchases.shape

True

### Selezionare solo i Nuovi Acquisti

- La nuova matrice riporta **tutti** gli acquisti, compresi quelli già indicati nella matrice precedente
- Vogliamo una matrice in cui siano riportati solo gli acquisti successivi all'analisi svolta sopra
- Possiamo ottenerla con una semplice differenza tra le due matrici

In [31]:
new_purchases = purchases_updated - purchases

- Quanti nuovi acquisti ha fatto mediamente ogni utente ?


In [32]:
mean_new_purchases = new_purchases.sum() / n_users
mean_new_purchases

32.98314606741573

In [33]:
np.median(new_purchases.sum(1)) # mediana

6.5

### Quali Nuovi Acquisti sono stati Suggeriti ?

- Abbiamo ora la matrice `suggestions` con gli acquisti _suggeriti_ e quella `new_purchases` con i nuovi acquisti _effettivi_
- Da queste possiamo individuare quali sono i suggerimenti **validi**, quelli a cui dopo l'analisi è corrisposto un acquisto
- Sono in pratica l'intersezione tra i due insiemi, che otteniamo moltiplicando le matrici elemento per elemento
  - ciascun valore della nuova matrice sarà 1 solo dove **entrambi** i valori delle due esistenti è 1

In [34]:
hits = suggestions * new_purchases

### Quanti Clienti hanno Ricevuto almeno un Suggerimento Valido ?

- Possiamo usare la funzione `max` per vedere in quali righe di `hits` sia presente almeno un 1, corrispondente ad un suggerimento valido

In [35]:
satisfied_users = hits.max(1)

- Quanti sono gli utenti che hanno ricevuto un suggerimento valido ?

In [36]:
satisfied_users.sum()

61

- ...e quanti sono in percentuale sul totale

In [37]:
satisfied_users.sum() / n_users

0.34269662921348315

- Abbiamo quindi previsto per il **34,3%** degli utenti almeno un prodotto che poi hanno acquistato
- ma quanto è buono questo risultato ?

### Confronto con una Selezione Casuale di Prodotti

- Per verificare il valore (o bontà) del risultato, misuriamo l'accuratezza ottenibile **suggerendo _N_ prodotti a caso** a ciascun utente
- Per estrarre dei suggerimenti casuali, definiamo una matrice con valori di interesse a caso tra 0 e 1 ...
  - il modulo `random` contiene le funzionalità relative alla generazione di array con valori casuali secondo varie distribuzioni
  - la funzione `seed` imposta il seed per la generazione di numeri casuali, in modo da rendere riproducibile il risultato
  - la funzione `random` genera un array di valori casuali con distribuzione uniforme in [0, 1) con forma data

In [38]:
np.random.seed(5)
random_interest = np.random.random(interest.shape)
random_interest[:5, :5]

array([[0.22199317, 0.87073231, 0.20671916, 0.91861091, 0.48841119],
       [0.16621773, 0.4949298 , 0.81012877, 0.78034263, 0.63087014],
       [0.36680512, 0.54382329, 0.07050355, 0.21437738, 0.78046745],
       [0.19330102, 0.67788327, 0.04413307, 0.06011549, 0.56624796],
       [0.34956196, 0.84851566, 0.08981507, 0.21130447, 0.84895548]])

- Come prima, impostiamo a 0 i valori per prodotti già acquistati...

In [39]:
random_interest[purchases_mask] = 0

- ...e selezioniamo gli _N_ prodotti di maggiore interesse per ogni utente

In [40]:
random_interest_ranking = (-random_interest).argsort(1).argsort(1)
random_suggestions = (random_interest_ranking < N).astype(np.int16)

- Rieseguiamo infine il confronto con i prodotti effettivamente acquistati

In [41]:
random_hits = random_suggestions * new_purchases
random_satisfied_users = random_hits.max(1)

In [42]:
random_satisfied_users.sum()

22

In [43]:
random_satisfied_users.sum() / n_users

0.12359550561797752

## Suggerimenti Casuali: Qual è l'Accuratezza Teorica ?
- il risultato del 12% appena calcolato è frutto di un singolo esperimento e cambia cambiando i suggerimenti casuali proposti ad ogni utente
 - basta cambiare il seed, ad es. `np.random.seed(2)` produce un'accuratezza del 15%   
- qual è allora il risultato corretto indipendente dal singolo esperimento ?   
- definiamo il problema utilizzando il calcolo delle probabilità 
 - qual è la probabilità che un prodotto scelto a caso sia di interesse per un utente ?
 - con $p$ prodotti in catalogo non ancora acquistati ed $i$ prodotti di interesse per ogni utente, allora è 
   - 1 - la prob. di non fornire nessun prodotto di interesse
 - considerando suggerimenti di $n$ prodotti, la prob. di NON fornire prodotti di interesse è 
 
$$\frac{\binom{p - i}{n}}{\binom{p}{n}}$$

- Nel nostro caso, assumiamo il numero di prodotti suggeribili per utente pari alla differenza tra quelli noti e quelli in media già acquistati

In [44]:
p = n_items - mean_purchases
p

3329.6011235955057

- Ipotizziamo che il numero di prodotti d'interesse per ciascun utente sia _i_ = 22
- In base a questo, la probabilità di azzeccare casualmente almeno un prodotto di interesse suggerendone 20 è ...

In [45]:
# importiamo la funzione che calcola il coefficiente binomiale
from scipy.special import binom
1 - binom(p-22 ,N) / binom(p ,N)

0.12450646886333139

## Conclusioni ##

- Proponendo suggerimenti di acquisto a caso, azzecchiamo gli acquisti futuri (periodo di test) per circa il **12%** degli utenti
 - identico di fatto al risultato teorico probabilistico del **12%** con $i=22$
- il metodo di recommendation esposto, basato sul semplice prodotto della matrice _utenti x prodotti_, è circa 3 volte più efficace di quello casuale
- ma come stimiamo il numero di prodotti di interesse, i.e. i = 22 ?
 - dal numero di acquisti che ipotizziamo facciano mediamente i clienti sulla base di quelli fatti in passato a parità di estensione temporale
 - essendo questo un esperimento volto a misurare quanto il risultato sia migliore di quello con suggerimenti casuali, $i$ è il num. di acquisti noti nel periodo di test  
 - tuttavia il numero medio di acquisti nel test è 33, ma con $i=33$ la prob. diventa 18%, perchè è più alta rispetto all'esperimento ?

## Esercizio: Calcoliamo la Probabilità Utente per Utente
- la probabilità teorica di suggerire casualmente prodotti validi è più alta dell'accuratezza ottenuta dal medesimo esperimento con $i=33$: 18% vs 12%, perchè ?
  - usare il semplice num. medio di acquisti implica assumere che questa probabilità aumenti linearmente rispetto ad $i$, ma ciò non è vero 
   - e.g. un utente col doppio degli acquisti di un altro, non ha una probabilità doppia di ricevere almeno un suggerimento di acquisto valido a parità di num. di suggerimenti
- **Esercizio**: calcolare questa probabilità utente per utente con $i$ diverso per ogni utente
 - la prob. teorica giusta è la media delle prob. calcolate per ciascun utente
 - il risultato è circa il 13%
 - Per verificarne la correttezza sperimentalmente, eseguire almeno un centinaio di simulazioni di suggerimenti di acquisti casuali e farne la media
 - per la legge dei grandi numeri, all'aumentare delle simulazioni quest'ultima media si avvicinerà sempre di più a quello esatto.